# Stimulus Response Analyses

In [1]:
%config Completer.use_jedi = False

In [2]:
from pynwb import NWBHDF5IO

## Load Data

In [3]:
# Load NWB file
io = NWBHDF5IO('nwb_local_data.nwb', 'r')
nwbfile = io.read()